## Installing and importing packages and softwares

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from matplotlib import rcParams
import matplotlib.colors as mcolors

import numpy as np
import distinctipy
from distinctipy import colorblind
import pandas as pd
import warnings

import anndata as ad
import scanpy as sc
import scipy

import seaborn as sns

import umap.umap_ as umap
#import umap.plot_ as umap_plot

from ipyfilechooser import FileChooser

#For Tables:
from IPython.display import display
from IPython.display import Latex
pd.set_option('display.max_columns', 500)
from tqdm import tnrange, tqdm_notebook
#%load_ext rpy2.ipython
%reload_ext rpy2.ipython


#Showing different spreads and min_dists - from https://scanpy.readthedocs.io/en/stable/tutorials/plotting/advanced.html
from itertools import product

## Import AnnData (adata) object that was converetd from a Seurat object made in Trailmaker

In [ ]:
adata = sc.read_h5ad("/Users/johnbriseno/Desktop/TMP_4_ZDrive/Data/snRNASeq/SAMap/h5ad/trailmaker_integrated_MWB_seurat_2_scanpy.h5ad")

In [ ]:
adata

## Downsample 10X nuclei and Parse nuclei to get 4000 for each sample
Used example from here: https://github.com/scverse/scanpy/issues/987

In [ ]:
target_cells = 3450 #number nuclei for each sample
cluster_key = "samples"

grouped = adata.obs.groupby(cluster_key)
downsampled_indices = []

for _, group in grouped:
    if len(group) > target_cells:
        downsampled_indices.extend(group.sample(target_cells).index)
    else:
        downsampled_indices.extend(group.index)

adata_downsampled = adata[downsampled_indices]

In [ ]:
adata_downsampled

## Looking at Trailmaker (Seurat) UMAP embeddings and changing some metadata

In [ ]:
sc.pl.umap(adata_downsampled, 
           color = ['seurat_clusters',
                    'samples'
                   ], 
           legend_loc = 'on data', legend_fontsize = 'xx-small', 
           ncols = 2,
           #save = '_trailmaker_seurat_Male_WBs_clusters2_UMAPs.svg'
          )

In [ ]:
#Changing sample color
sc.pl.umap(adata_downsampled, 
           color = ['samples'], 
           legend_loc = 'on data', legend_fontsize = 'xx-small', 
           ncols = 2,
           palette={"MZ7": "tab:blue", "MA7": "tab:purple"},
           #save = '_Male_WBs_clusters2_UMAPs.svg'
          )

## Rerun Analysis in Scanpy
With help from this tutorial: https://nbisweden.github.io/workshop-scRNAseq/labs/scanpy/scanpy_07_trajectory.html

In [ ]:
#first, store the old umap with a new name so it is not overwritten
#adata_downsampled.obsm['X_umap_old'] = adata_downsampled.obsm['X_umap'] #-> only run this once
sc.pp.neighbors(adata_downsampled, 
                n_pcs = 40, #same PCs as trailamker
                n_neighbors = 20, #what was trialmaker run with?
                use_rep="X_harmony") #we used 4000 HVGs
sc.tl.umap(adata_downsampled, min_dist=0.4, spread=3)

In [ ]:
# Redo clustering as well
#sc.tl.leiden(adata_downsampled, key_added = "leiden_0.8", resolution = 0.8) 
#sc.tl.leiden(adata_downsampled, key_added = "leiden_1.0", resolution = 1.0)
#sc.tl.leiden(adata_downsampled, key_added = "leiden_1.2", resolution = 1.2)
#sc.tl.leiden(adata_downsampled, key_added = "leiden_1.4", resolution = 1.4)

sc.pl.umap(adata_downsampled, color = ['seurat_clusters', 'leiden_0.8', 'leiden_1.0', 'leiden_1.2', 'leiden_1.4'],
           legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =1,
           #save="_scanpy_clusters_various_resolutons_n_neighbor_20.png"
          )

#### saving clustering so we can just run from here next time

In [ ]:
adata_downsampled.write("/Users/johnbriseno/Desktop/TMP_4_ZDrive/Data/snRNASeq/SAMap/h5ad/adata_downsampled_trailmaker_integrated_MWB_seurat_2_scanpy.h5ad")

In [ ]:
#reading back in h5ad
adata_downsampled = sc.read_h5ad("/Users/johnbriseno/Desktop/TMP_4_ZDrive/Data/snRNASeq/SAMap/h5ad/adata_downsampled_trailmaker_integrated_MWB_seurat_2_scanpy.h5ad")

In [ ]:
# plot new scanpy clusters onto the seurat embedding:
sc.pl.embedding(adata_downsampled, basis='X_umap_old', color = ['seurat_clusters','leiden_0.8', 'leiden_1.0', 'leiden_1.2', 'leiden_1.4'],
                legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =1,
               save="_scanpy_clusters__n.neighbor20_on_trailmaker_Serurat_embeddings.png"
               )

In [ ]:
sc.tl.umap(adata_downsampled, min_dist=0.4, spread=3, n_components=3) #making these 3d off the bat
sc.pl.umap(adata_downsampled, color = 'seurat_clusters', projection='3d', save="3dumap_seurat_clusters.png")
sc.pl.umap(adata_downsampled, color = 'leiden_0.8', projection='3d', save="3dumap_leiden_0.8.png")
sc.pl.umap(adata_downsampled, color = 'leiden_1.0', projection='3d', save="3dumap_leiden_1.0.png")
sc.pl.umap(adata_downsampled, color = 'leiden_1.2', projection='3d', save="3dumap_leiden_1.2.png")
sc.pl.umap(adata_downsampled, color = 'leiden_1.4', projection='3d', save="3dumap_leiden_1.4.png")

## Checkout clusters from sample
with frequency plot example from here: https://www.biostars.org/p/9575906/

In [ ]:
color = ['purple', 'blue']

tmp = pd.crosstab(adata_downsampled.obs['seurat_clusters'],adata.obs['samples'], normalize='index')
tmp.plot.bar(stacked=True).legend(loc='upper right')
tmp = pd.crosstab(adata_downsampled.obs['leiden_0.8'],adata.obs['samples'], normalize='index')
tmp.plot.bar(stacked=True).legend(loc='upper right')
tmp = pd.crosstab(adata_downsampled.obs['leiden_1.0'],adata.obs['samples'], normalize='index')
tmp.plot.bar(stacked=True).legend(loc='upper right')
tmp = pd.crosstab(adata_downsampled.obs['leiden_1.2'],adata.obs['samples'], normalize='index')
tmp.plot.bar(stacked=True).legend(loc='upper right')
tmp = pd.crosstab(adata_downsampled.obs['leiden_1.4'],adata.obs['samples'], normalize='index')
tmp.plot.bar(stacked=True).legend(loc='upper right')

In [ ]:
#getting colors -> https://stackoverflow.com/questions/11927715/how-to-give-a-pandas-matplotlib-bar-graph-custom-colors
#saving fig -> https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html
#getting counts, not relative percentages -> get rid of normalize='columns' from pd.crosstab(y,x, nomralize = 'columns')


tmp = pd.crosstab(adata_downsampled.obs['seurat_clusters'],adata_downsampled.obs['samples']).T.plot(kind='bar', stacked=True, color=adata_downsampled.uns['seurat_clusters_colors'])
tmp.legend(title='seurat_clusters', bbox_to_anchor=(1.26, 1.02),loc='upper right',labelcolor=adata_downsampled.uns['seurat_clusters_colors'])
plt.savefig('/Users/johnbriseno/Desktop/M_WBs_Jupyter/figures/seurat_clusters_freq_plot.png', 
            bbox_inches='tight', pad_inches=0.1,
            dpi=400, format='png')

tmp = pd.crosstab(adata_downsampled.obs['leiden_0.8'],adata_downsampled.obs['samples']).T.plot(kind='bar', stacked=True, color=adata_downsampled.uns['leiden_0.8_colors'])
tmp.legend(title='leiden_0.8', bbox_to_anchor=(1.26, 1.02),loc='upper right',labelcolor=adata_downsampled.uns['leiden_0.8_colors'])
plt.savefig('/Users/johnbriseno/Desktop/M_WBs_Jupyter/figures/leiden_0.8_freq_plot.png', 
            bbox_inches='tight', pad_inches=0.1,
            dpi=400, format='png')

tmp = pd.crosstab(adata_downsampled.obs['leiden_1.0'],adata_downsampled.obs['samples']).T.plot(kind='bar', stacked=True, color=adata_downsampled.uns['leiden_1.0_colors'])
tmp.legend(title='leiden_1.0', bbox_to_anchor=(1.26, 1.02),loc='upper right',labelcolor=adata_downsampled.uns['leiden_1.0_colors'])
plt.savefig('/Users/johnbriseno/Desktop/M_WBs_Jupyter/figures/leiden_1.0_freq_plot.png', 
            bbox_inches='tight', pad_inches=0.1,
            dpi=400, format='png')

tmp = pd.crosstab(adata_downsampled.obs['leiden_1.2'],adata_downsampled.obs['samples']).T.plot(kind='bar', stacked=True, color=adata_downsampled.uns['leiden_1.2_colors'])
tmp.legend(title='leiden_1.2', bbox_to_anchor=(1.26, 1.02),loc='upper right',labelcolor=adata_downsampled.uns['leiden_1.2_colors'])
plt.savefig('/Users/johnbriseno/Desktop/M_WBs_Jupyter/figures/leiden_1.2_freq_plot.png', 
            bbox_inches='tight', pad_inches=0.1,
            dpi=400, format='png')

tmp = pd.crosstab(adata_downsampled.obs['leiden_1.4'],adata_downsampled.obs['samples']).T.plot(kind='bar', stacked=True, color=adata_downsampled.uns['leiden_1.4_colors'])
tmp.legend(title='leiden_1.4', bbox_to_anchor=(1.26, 1.02),loc='upper right',labelcolor=adata_downsampled.uns['leiden_1.4_colors'])
plt.savefig('/Users/johnbriseno/Desktop/M_WBs_Jupyter/figures/leiden_1.4_freq_plot.png', 
            bbox_inches='tight', pad_inches=0.1,
            dpi=400, format='png')

#do this but have raw count? - normalize='columns' from pd.crosstab(y,x, nomralize = 'columns')
#need to fix legends 

## Ranking genes to find cluster-specific markers and across resolutions
https://scanpy-tutorials.readthedocs.io/en/multiomics/visualizing-marker-genes.html#Visualization-of-marker-genes

https://github.com/scverse/scanpy/issues/748 <- good explanation of marker genes, 
such that "tells you which genes characterize a cluster, but doesn't necessarily tell you 
which genes contributed most to the global split of clusters that was generated

With these, can do marker gene visualization downstream.

Let's just get unique genes of interest too (must filter out adata which is just a df)

In [ ]:

#must call all rankings by their own names, otherwise just rewriting a bunch of stuff

sc.tl.rank_genes_groups(adata_downsampled, 
                        groupby="seurat_clusters",
                        method="wilcoxon",
                        corr_method="benjamini-hochberg", 
                        key_added="wilcoxon_seurat_clusters")
#Apply more stringent filter
sc.tl.filter_rank_genes_groups(adata_downsampled,
                               groupby="seurat_clusters",
                               key='wilcoxon_seurat_clusters',
                               min_fold_change=1,
                               key_added='wilcoxon_filtered_seurat_clusters')


sc.tl.rank_genes_groups(adata_downsampled, 
                        groupby="leiden_0.8",
                        method="wilcoxon",
                        corr_method="benjamini-hochberg", 
                        key_added="wilcoxon_leiden_0.8")
#Apply more stringent filter
sc.tl.filter_rank_genes_groups(adata_downsampled,
                               groupby="leiden_0.8",
                               key='wilcoxon_leiden_0.8',
                               min_fold_change=1,
                               key_added='wilcoxon_filtered_leiden_0.8')


sc.tl.rank_genes_groups(adata_downsampled, 
                        groupby="leiden_1.0",
                        method="wilcoxon",
                        corr_method="benjamini-hochberg", 
                        key_added="wilcoxon_leiden_1.0")
#Apply more stringent filter
sc.tl.filter_rank_genes_groups(adata_downsampled,
                               groupby="leiden_1.0",
                               key='wilcoxon_leiden_1.0',
                               min_fold_change=1,
                               key_added='wilcoxon_filtered_leiden_1.0')


sc.tl.rank_genes_groups(adata_downsampled, 
                        groupby="leiden_1.2",
                        method="wilcoxon",
                        corr_method="benjamini-hochberg", 
                        key_added="wilcoxon_leiden_1.2")
#Apply more stringent filter
sc.tl.filter_rank_genes_groups(adata_downsampled,
                               groupby="leiden_1.2",
                               key='wilcoxon_leiden_1.2',
                               min_fold_change=1,
                               key_added='wilcoxon_filtered_leiden_1.2')

sc.tl.rank_genes_groups(adata_downsampled, 
                        groupby="leiden_1.4",
                        method="wilcoxon",
                        corr_method="benjamini-hochberg", 
                        key_added="wilcoxon_leiden_1.4")
#Apply more stringent filter
sc.tl.filter_rank_genes_groups(adata_downsampled,
                               groupby="leiden_1.4",
                               key='wilcoxon_leiden_1.4',
                               min_fold_change=1,
                               key_added='wilcoxon_filtered_leiden_1.4')


#annot

In [ ]:
#running dendrogram to cluster clusters based on similarity 
sc.tl.dendrogram(adata_downsampled, groupby="seurat_clusters")
sc.tl.dendrogram(adata_downsampled, groupby="leiden_0.8")
sc.tl.dendrogram(adata_downsampled, groupby="leiden_1.0")
sc.tl.dendrogram(adata_downsampled, groupby="leiden_1.2")
sc.tl.dendrogram(adata_downsampled, groupby="leiden_1.4")

#checkout heatmaps from diff clusterings with 'rank_genes_groups'
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='seurat_clusters',
    n_genes=3,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_seurat_clusters',
    swap_axes=True,
    save='_wilcoxon_seurat_clusters.png'
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_0.8',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_leiden_0.8',
    swap_axes=True,
    save='_wilcoxon_leiden_0.8.png'
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_1.0',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_leiden_1.0',
    swap_axes=True,
    save='_wilcoxon_leiden_1.0.png',
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_1.2',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_leiden_1.2',
    swap_axes=True,
    save='_wilcoxon_leiden_1.2.png',
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_1.4',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_leiden_1.4',
    swap_axes=True,
    save='_wilcoxon_leiden_1.4.png',
)
#annot

In [ ]:
#checkout heatmaps from diff clusterings with 'rank_genes_groups'
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='seurat_clusters',
    n_genes=3,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_filtered_seurat_clusters',
    swap_axes=True,
    save='_wilcoxon_filtered_seurat_clusters.png'
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_0.8',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_filtered_leiden_0.8',
    swap_axes=True,
    save='_wilcoxon_filtered_leiden_0.8.png'
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_1.0',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_filtered_leiden_1.0',
    swap_axes=True,
    save='_wilcoxon_filtered_leiden_1.0.png',
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_1.2',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_filtered_leiden_1.2',
    swap_axes=True,
    save='_wilcoxon_filtered_leiden_1.2.png',
)
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_1.4',
    n_genes=5,
    show_gene_labels=True,
    cmap="viridis",
    key='wilcoxon_filtered_leiden_1.4',
    swap_axes=True,
    save='_wilcoxon_filtered_leiden_1.4.png',
)

In [ ]:
#getting just unique markers per cluster
#code from google ai agh 

#basically extract adata as dataframe and filter for just genes in cluster

###Step 1: Run sc.tl.rank_genes_groups() and name each key independtly (did up top)

In [ ]:
###Step 2: Extract the results into a pandas DataFrame
results_df = sc.get.rank_genes_groups_df(adata_downsampled, group=None, key='wilcoxon_leiden_1.0')
#group=None: This parameter tells the function to get the results for all groups, which is what you need to filter for uniqueness.
#Must specify key since there is no 'rank_genes_groups' default - each DE filter is called by its resolution

In [ ]:
# Define a function to get unique markers
def get_unique_markers(results_df, n_genes):
    all_genes = []
    unique_markers = {}
    
    # Get the list of clusters from the DataFrame
    clusters = results_df['group'].unique()
    
    for cluster in clusters:
        # Get top N genes for the current cluster
        cluster_genes = results_df[results_df['group'] == cluster].head(n_genes)['names'].tolist()
        
        # Keep genes not yet seen in previous clusters
        unique_genes = [gene for gene in cluster_genes if gene not in all_genes]
        unique_markers[cluster] = unique_genes
        
        # Add the unique genes to the master list
        all_genes.extend(unique_genes)
        
    return unique_markers

In [ ]:
# Get the top 5 unique markers for each cluster
n_genes_per_cluster = 5
unique_markers_dict = get_unique_markers(results_df, n_genes_per_cluster)

# Print the resulting dictionary of unique markers
for cluster, genes in unique_markers_dict.items():
    print(f"Cluster {cluster}: {genes}")

In [ ]:
sc.pl.dotplot(adata_downsampled, 
              #unique_markers_dict, 
              groupby='leiden_1.0',
              dendrogram=True,
              swap_axes=True,
              #save="_unique_markers_leiden_1.0_dotplot.svg"
             )

## Also try with really low resolutions to checkout 
Resolution is largely arbitrary but want to make sure we have strong cluster-specific markers

In [ ]:
#also try with really low resolutions to checkout 
sc.tl.leiden(adata_downsampled, key_added = "leiden_0.02", resolution = 0.02) # default resolution in 0.8
sc.tl.leiden(adata_downsampled, key_added = "leiden_0.05", resolution = 0.05) # default resolution in 1.0
sc.tl.leiden(adata_downsampled, key_added = "leiden_0.075", resolution = 0.075) # default resolution in 1.2
sc.tl.leiden(adata_downsampled, key_added = "leiden_0.1", resolution = 0.1) # default resolution in 1.4
sc.tl.leiden(adata_downsampled, key_added = "leiden_0.5", resolution = 0.5) # default resolution in 1.4

In [ ]:
sc.pl.umap(adata_downsampled, color = ['leiden_0.02', 'leiden_0.05', 'leiden_0.075', 'leiden_0.1', 'leiden_0.5'],legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =2)
#looks like only 0.5 looks reasonable

In [ ]:
#frequency plots
tmp = pd.crosstab(adata_downsampled.obs['leiden_0.5'],adata.obs['samples'], normalize='index')
tmp.plot.bar(stacked=True).legend(loc='upper right')

In [ ]:
#markeger genes
sc.tl.rank_genes_groups(adata_downsampled, 
                        groupby="leiden_0.5",
                        groups='all',
                        reference='rest',
                        method="wilcoxon",
                        corr_method="benjamini-hochberg", key_added="rank_genes_groups")
#Apply more stringent filter
sc.tl.filter_rank_genes_groups(adata_downsampled,
                               groupby="leiden_0.5",
                               min_in_group_fraction=0.25,
                               max_out_group_fraction=0.5,
                               min_fold_change=1, key_added='rank_genes_groups_filtered')

In [ ]:
sc.pl.rank_genes_groups_heatmap(
    adata_downsampled,
    groupby='leiden_0.5',
    n_genes=10,
    show_gene_labels=True,
    cmap="viridis",
    key='rank_genes_groups_filtered',
    swap_axes=True
)
sc.pl.rank_genes_groups_stacked_violin(
    adata_downsampled,
    groupby='leiden_0.5',
    n_genes=10,
    cmap="viridis",
    key='rank_genes_groups_filtered',
)

#### Now pick a resolution to run on downstream analyses

## Visualizing known marker genes

In [ ]:
#GOI plots
#Setting gene directories
markers_genes_dict = {
    'Putative Hematopoietic Markers':['g29365', #HHEX
                                      'g21408', #MARCO
                                      'g24607', #ssc4D
                                      'g25995' #hephl1
                                     ],
    'WB Homolog Single-Cell Markers': ['g776', #SVEP1
                                       'g19257', #HSPG2
                                       'g13641', #PCNA
                                       'g8344' #gata3
                                      ],
    'Hemocyte Markers': ['cluster-2407', #EsPGRP5
                         'g2715', #EsTLR
                         'g4518', #Esgalectin2
                         'g18324' #myd88
                        ]
}
type(markers_genes_dict)

In [ ]:
markers_list = [
    #wb bulk markers
    'g29365', #HHEX
    'g21408', #MARCO
    'g24607', #ssc4D
    'g25995', #hephl1

    #wb sc markers
    'g776', #SVEP1
    'g19257', #HSPG2
    'g13641', #PCNA
    'g8344', #gata3
    
    #hct markers
    'cluster-2407', #EsPGRP5
    'g2715', #EsTLR
    'g4518', #Esgalectin2
    'g18324' #myd88
          ]
type(markers_list)

In [ ]:
#gotta remake umap as 2d
#sc.tl.umap(adata_downsampled, min_dist=0.4, spread=3)

sc.pl.umap(adata_downsampled, color = markers_list, 
           ncols= 2, cmap=sns.cubehelix_palette(dark=0, light=.9, as_cmap=True))
#use_raw = False -> raw genes (adata.raw.var_names) are those ~39K genes not in the 2000 HVGs used to integrate the data (which are in adata.var_names)
#lowly expressed genes (ie HHEX) are not HVGs and thus in raw
#cmap from here:https://github.com/scverse/scanpy/issues/1550

In [ ]:
#From stakced violin tutorial:
#https://scanpy.readthedocs.io/en/stable/generated/scanpy.pl.stacked_violin.html

#How to save these matplotlib figs with add_totals():
#https://stackoverflow.com/questions/78463183/problem-exporting-images-with-matplot-lib

sc.tl.dendrogram(adata_downsampled, groupby = 'leiden_1.0')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10), gridspec_kw={"wspace": 0.2})

ax1_dict = sc.pl.dotplot(
    adata_downsampled, markers_genes_dict, groupby="leiden_1.0", ax=ax1, var_group_rotation=45,#, swap_axes= True,
    show=False, 
    dendrogram = True)

vp = sc.pl.stacked_violin(
    adata_downsampled, markers_genes_dict, groupby="leiden_1.0", ax=ax2, var_group_rotation=45, swap_axes= True,
    show=True, 
    return_fig = True)
vp.add_totals(sort = 'ascending', color= adata_downsampled.uns['leiden_1.0_colors']).style(ylim=(0,5)).show()


#fig.savefig("/Users/johnbriseno/Desktop/M_WBs_Jupyter/figures/WB_GOIs.svg", format="svg")
fig.show()


#annot
#seurat_clusters

#### Another GOI dictionary for model figure

In [ ]:
#GOI plots
#Setting gene directories
funct_marker_genes_dict = {
    'MTC Membrane Transporters':['g15814', #MTFR1
                                 'g3979', #SLC25A31
                                 'g28362' #VDAC2
                                ],
    'Transcription Factors':['g29365', #HHEX
                             'g8344', #GATA3
                             'g15292' #SOX7
                            ],
    'RNA Transport & Stability':[#'g16209', #HNRNPA2B1
                                 'g16208', #HNRNPA2B1
                                 'g17243', #HNRNPA2B1
                                 'g13639' #HNRNPH1
                                ],
    'Actin-Mediated Cytoplasmic Bridge':['g2950', #Tm1
                                        'g18044', #FSCN1
                                        'g10567' #COTL1
                                        ],
    'Ras GTPase Signaling':['g14463', #ARHGDIA
                           'g18872', #RHOA
                           'g26916' #RABGAP1
                           ],
    'Ion Binding & Transport':['g25995', #HEPHL1
                              'g29902', #FTH1
                              'g27264' #SPTAN1
                              ],
    'Pattern Recognition Receptors':['cluster-2407', #EsPGRP5
                                     'g21408', #MARCO
                                     'g24607' #SSC4D
                                    ]
}
type(funct_marker_genes_dict)

In [ ]:
sc.pl.stacked_violin(adata_downsampled, 
                     funct_marker_genes_dict, groupby="leiden_1.0", 
                     dendrogram = True,
                     var_group_rotation=45, swap_axes= True,
                     cmap='viridis',
                    save = '_violin_model_GOIs.svg')

## Run PAGA  on these new nuclei embeddings and resolution
Use the clusters from leiden clustering run PAGA. First we create the graph and initialize the positions using the umap.

In [ ]:
# use the umap to initialize the graph layout.
sc.tl.draw_graph(adata_downsampled, init_pos='X_umap')

#use force atlas 2 to spatialize clusters on umap embeddings
sc.pl.draw_graph(adata_downsampled, color='leiden_1.0',
                 legend_loc='on data', 
                 legend_fontsize = 'small',
                #save='_n_neighbor20_UMAP_embedding_FA.png'
                )

#plot paga map
sc.tl.paga(adata_downsampled, groups='leiden_1.0')
sc.pl.paga(adata_downsampled, color='leiden_1.0', 
           edge_width_scale = 0.7, 
           node_size_scale=3,
           #save='_n_neighbor20_UMAP_embedding_PAGA.png'
          )

#### Lines between course-grained paga clusters represents relative relatedness (i.e. how many genes are shared between clusters)

## Filtering graph edges
First, lets explore the graph a bit. So we plot the umap with the graph connections on top.

In [ ]:
sc.pl.umap(adata_downsampled, edges=True, color = 'leiden_1.0', 
           legend_loc= 'on data', legend_fontsize= 'xx-small',
           #save="_leiden_1.0_20n.neighbors_edges.png"
          )

#### We have many edges in the graph between unrelated clusters, so lets try with fewer neighbors.

In [ ]:
sc.pp.neighbors(adata_downsampled, n_neighbors=5, n_pcs = 40) #ran 40pcs before
sc.pl.umap(adata_downsampled, edges=True, color = 'leiden_1.0', 
           legend_loc= 'on data', legend_fontsize= 'xx-small',
           #save="_leiden_1.0_5n.neighbors_edges.png"
          )

## Rerun PAGA after lowering n_neighbors

In [ ]:
#use the umap to initialize the graph layout and use FA2 to spatialize
#sc.tl.umap(adata_downsampled, min_dist=0.4, spread=3)
#sc.tl.draw_graph(adata_downsampled, init_pos='X_umap')
sc.pl.draw_graph(adata_downsampled, color='leiden_1.0', legend_loc='on data', 
                 legend_fontsize=  'xx-small', edges = True, #save = '_n_neighbor5_UMAP_embedding_FA.png'
                )

#then rerun paga
#sc.tl.paga(adata_downsampled, groups='leiden_1.0')
sc.pl.paga(adata_downsampled, color='leiden_1.0', edge_width_scale = 0.5, #node_size_scale= 2, save='_n_neighbor5_UMAP_embedding_PAGA.png'
          )

## Embedding using PAGA-initialization
We can now redraw the graph using the paga layout itself. The following is just as well possible for a UMAP or FA2 layout.

In [ ]:
sc.tl.draw_graph(adata_downsampled, init_pos='paga')

Now we can see all marker genes also at single-cell resolution in the paga-oriented layout.

In [ ]:
sc.pl.umap(adata_downsampled, color=['annot'], legend_loc='on data', legend_fontsize=  'xx-small', edges= True), #save='_paga_embed_n_neighbor5_umap.png')

In [ ]:

sc.pl.draw_graph(adata_downsampled, color=['leiden_1.0'], legend_loc='on data', legend_fontsize=  'xx-small', edges= True, save='_paga_embed_n_neighbor5_fa.png')

In [ ]:
sc.pl.paga(adata_downsampled, color='leiden_1.0', edge_width_scale = 0.5, node_size_scale= 2, save='_n_neighbor5_paga_embed_paga.png')

#### Compare UMAP and PAGA based on PAGA embedding

In [ ]:
sc.pl.paga_compare(
    adata_downsampled, threshold=0.05, title='leiden_1.0',
    size=10, edge_width_scale=0.5,
    legend_fontsize=12, fontsize=12, frameon=True, edges=True, save='_paga_fa_compare_leiden_1.0.png')

#### testing 11/22/2025

In [ ]:
#DO NOT RUN: #nothing too convincing here...
#probably not the best data to try this since there is only 2 biological replicates that were sequenced differently

#Rename clusters with really clear markers that we ID's from our bulk analyses and from previous 'omics papers

#for leiden_1.0 annots
annot = pd.DataFrame(adata_downsampled.obs['leiden_1.0'].astype('string'))
annot[annot['leiden_1.0'] == '9'] = '9_hyp_hcb' #hypothetical hematopoietic stem cells based on HHEX and other marker expression
adata_downsampled.obs['annot']=annot['leiden_1.0'].astype('category')
sc.pl.umap(adata_downsampled, color = 'annot',legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =2)
annot.value_counts()
type(annot)


#Pseudotime with Monocle and hopping back to the first tutorial:
#https://nbisweden.github.io/workshop-scRNAseq/labs/scanpy/scanpy_07_trajectory.html)

#We can reconstruct gene changes along PAGA paths for a given set of genes
#Choose a root cell for diffusion pseudotime.

#docs say to first run diffmap then do dtp
#https://scanpy.readthedocs.io/en/stable/generated/scanpy.tl.dpt.html
#https://scanpy.readthedocs.io/en/stable/generated/scanpy.tl.diffmap.html#scanpy.tl.diffmap


#need to use gauss and not umap - In order to reproduce the original implementation of DPT, use method=='gauss'. Using the default method=='umap' only leads to minor quantitative differences, though.
sc.pp.neighbors(adata_downsampled, n_neighbors = 5, #use latest settings
                n_pcs = 40, method = 'gauss', use_rep = "X_harmony")
adata_downsampled.uns['iroot'] = np.flatnonzero(adata_downsampled.obs['annot']  == '9_hyp_hcb')[0]


#As previously, dpt() came with a default parameter of n_dcs=10 but diffmap() has a default parameter of n_comps=15, 
#you need to pass n_comps=10 in diffmap() in order to exactly reproduce previous dpt() results. 
sc.tl.diffmap(adata_downsampled, n_comps=10),
                  #neighbors_key=None, random_state=0, copy=False)

sc.tl.dpt(adata_downsampled)#, n_dcs=10, n_branchings=0, 
              #min_group_size=0.01, allow_kendall_tau_shift=True, 
              #neighbors_key=None, copy=False)


sc.pl.draw_graph(adata_downsampled, color=['annot', 'dpt_pseudotime'], legend_loc='on data', legend_fontsize= 'x-small')

#am able to get pseudotime to run but without canonical markers to clusters it's moot

In [ ]:
sc.pl.correlation_matrix(adata_downsampled, groupby="leiden_1.0")

In [ ]:
sc.pl.paga(adata_downsampled, layout='rt', root=[9])

In [ ]:
sc.pl.highest_expr_genes(adata_downsampled, n_top=30)
sc.pl.highly_variable_genes(adata_downsampled)

In [ ]:
sc.pl.diffmap(adata_downsampled, color = 'leiden_1.0')

## Further pruning PAGA edges 
Again from this tutorial -> https://scanpy.readthedocs.io/en/latest/tutorials/plotting/advanced.html

Remove weak PAGA edges in the plot based on edge weight distribution.

This is based on the assumption that most edge weights will be relatively weak and that we can therefore spot the few most interesting edges as the outliers of the distribution.

In [ ]:
# Distribution of PAGA connectivities for determining the cutting threshold
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
paga_conn = adata_downsampled.uns["paga"]["connectivities"].toarray().ravel()
a = axs[0].hist(paga_conn, bins=30)
sns.violinplot(paga_conn, ax=axs[1], inner=None)
sns.swarmplot(paga_conn, ax=axs[1], color="k", size=1)
thr = .5
_ = axs[1].axhline(thr, c="r")
_ = axs[0].axvline(thr, c="r")
#fig.savefig("/Users/johnbriseno/Desktop/M_WBs_Jupyter/figures/M_WBs_paga_pruning_qc.png", format="png")
fig.show()

In [ ]:
# Compare PAGA with and without prunning
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
sc.pl.paga(adata_downsampled, ax=axs[0], title="PAGA", show=False)
sc.pl.paga(adata_downsampled, ax=axs[1], title="PAGA - prunned", threshold=thr, save = 'paga_pruning.png')

## PAGA super-imposed on UMAP and FA
The layout used in PAGA is optimised to correspond to the PAGA connectivties (edge weighs). However, sometimes we would wish to have a different layout. For this we can use the pos argument.

In [ ]:
# Compare UMAP and PAGA layouts
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
sc.pl.umap(adata_downsampled, color="leiden_1.0", ax=axs[0], show=False, title="UMAP", legend_loc="on data")
sc.pl.paga(adata_downsampled, ax=axs[1], title="PAGA")

#### PAGA layout corresponding to UMAP
Set PAGA dot centers to the mean of the UMAP embedding values of cells from the corresponding groups.

In [ ]:
# Define PAGA positions based on the UMAP layout -
#for each cluster we use the mean of the UMAP positions from the cells in that cluster
pos = pd.DataFrame(adata_downsampled.obsm["X_umap"], index=adata_downsampled.obs_names)
pos["group"] = adata_downsampled.obs[adata_downsampled.uns["paga"]["groups"]]
pos = pos.groupby("group", observed=True).mean()

# Plot UMAP in the background
ax = sc.pl.umap(adata_downsampled, show=False)
# Plot PAGA ontop of the UMAP
with rc_context({"figure.figsize": (5, 5)}):
    sc.pl.paga(
        adata_downsampled,
        color="leiden_1.0",
        threshold=thr,
        node_size_scale=2,
        edge_width_scale=.5,
        pos=pos.values,
        random_state=0,
        ax=ax,
        frameon=False,
        #save='paga_on_umap.svg'
)

In [ ]:
# Define PAGA positions based on the FA layout -
#for each cluster we use the mean of the fa positions from the cells in that cluster
pos = pd.DataFrame(adata_downsampled.obsm["X_draw_graph_fa"], index=adata_downsampled.obs_names)
pos["group"] = adata_downsampled.obs[adata_downsampled.uns["paga"]["groups"]]
pos = pos.groupby("group", observed=True).mean()

# Plot fa in the background
ax = sc.pl.draw_graph(adata_downsampled, show=False)
# Plot PAGA ontop of the UMAP
sc.pl.paga(
    adata_downsampled,
    color="leiden_1.0",
    threshold=thr,
    node_size_scale=1,
    edge_width_scale=0.5,
    pos=pos.values,
    random_state=0,
    ax=ax,
    frameon=False,
    save='paga_on_fa.svg'

)

#### Showing different spreads and min_dists for UMAP
from: https://scanpy.readthedocs.io/en/stable/tutorials/plotting/advanced.html

In [ ]:
# Copy adata not to modify UMAP in the original adata object
adata_temp = adata_downsampled.copy()
# Loop through different umap parameters, recomputting and replotting UMAP for each of them
MIN_DISTS = [0.1, 1, 2]
SPREADS = [0.5, 1, 5]

# Create grid of plots, with a little extra room for the legends
fig, axes = plt.subplots(
    len(MIN_DISTS), len(SPREADS), figsize=(len(SPREADS) * 3 + 2, len(MIN_DISTS) * 3)
)
for (i, min_dist), (j, spread) in product(enumerate(MIN_DISTS), enumerate(SPREADS)):
    ax = axes[i][j]
    param_str = " ".join(["min_dist =", str(min_dist), "and spread =", str(spread)])
    # Recompute UMAP with new parameters
    sc.tl.umap(adata_temp, min_dist=min_dist, spread=spread)
    # Create plot, placing it in grid
    sc.pl.umap(
        adata_temp,
        color=["leiden_1.0"],
        title=param_str,
        s=40,
        ax=ax,
        show=False,
    )
plt.tight_layout()
plt.show()

fig.savefig("_paga_embed_umap_diff_parameters.png", format="png")
#fig.show()
plt.close()
del adata_temp

#annot
#seurat_clusters

#### Producing nicer UMAPs/FA with PAGA embedding

In [ ]:
#sc.tl.draw_graph(adata_downsampled, init_pos='umap')

#with rc_context({"figure.figsize": (5, 5)}):
sc.pl.umap(
        adata_downsampled,
        color="leiden_1.0",
        add_outline=True,
        legend_loc="on data",
        legend_fontsize=10,
        legend_fontoutline=2,
        frameon=False,
        title='',
        edges=True,
        save='_n_neighbor5_paga_embed_umap.svg'
    )

In [ ]:
sc.pl.umap(
        adata_downsampled,
        color="leiden_1.0",
        add_outline=True,
        legend_loc="on data",
        legend_fontsize=10,
        legend_fontoutline=2,
        frameon=False,
        title='',
        edges=True,
        save='_n_neighbor5_paga_embed_umap.svg'
    )

In [ ]:
#sc.tl.draw_graph(adata_downsampled, init_pos='X_draw_graph_fa')

#with rc_context({"figure.figsize": (5, 5)}):
sc.pl.draw_graph(
        adata_downsampled,
        color="leiden_1.0",
        add_outline=True,
        legend_loc="on data",
        legend_fontsize=10,
        legend_fontoutline=2,
        frameon=False,
        edges=True,
        title='',
        save='_n_neighbor5_paga_embed_fa.svg',
    )

In [ ]:
axes_dict = vp.get_axes()
print(axes_dict)

#### assigning clusters based on bulk markers as evidence - we did not run in our analyses

In [ ]:
#Assigning names to clusters with really clear markers that we ID's from our bulk analyses and from previous 'omics papers
#for leiden_1.0 annots
annot = pd.DataFrame(adata.obs['leiden_1.0'].astype('string'))
annot[annot['leiden_1.0'] == '8'] = '8_hyp_hsc' #hypothetical hematopoietic stem cells based on HHEX and other marker expression
adata.obs['annot']=annot['leiden_1.0'].astype('category')
sc.pl.umap(adata, color = 'annot',legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =2)
annot.value_counts()
type(annot)

#for seurat_clusters annots
annot = pd.DataFrame(adata.obs['seurat_clusters'].astype('string'))
annot[annot['seurat_clusters'] == '6'] = '6_hyp_hsc' #hypothetical hematopoietic stem cells based on HHEX and other marker expression
adata.obs['annot']=annot['seurat_clusters'].astype('category')
sc.pl.umap(adata, color = 'annot',legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =2)
annot.value_counts()
type(annot)

We can run monocle for pseudotime, but only once we start seeing good trends in data. Our data is OK but not seeing too much relevancy. 

In [ ]:
#DO NOT RUN: #nothing too convincing here...
#probably not the best data to try this since there is only 2 biological replicates that were sequenced differently

#Pseudotime with Monocle and hopping back to the first tutorial:
#https://nbisweden.github.io/workshop-scRNAseq/labs/scanpy/scanpy_07_trajectory.html)

#We can reconstruct gene changes along PAGA paths for a given set of genes
#Choose a root cell for diffusion pseudotime.

#docs say to first run diffmap then do dtp
#https://scanpy.readthedocs.io/en/stable/generated/scanpy.tl.dpt.html
#https://scanpy.readthedocs.io/en/stable/generated/scanpy.tl.diffmap.html#scanpy.tl.diffmap


#need to use gauss and not umap - In order to reproduce the original implementation of DPT, use method=='gauss'. Using the default method=='umap' only leads to minor quantitative differences, though.
sc.pp.neighbors(adata, n_neighbors = 5, #use latest settings
                n_pcs = 41, method = 'gauss', use_rep = "X_harmony")
adata.uns['iroot'] = np.flatnonzero(adata.obs['annot']  == '8_hyp_hsc')[0]


#As previously, dpt() came with a default parameter of n_dcs=10 but diffmap() has a default parameter of n_comps=15, 
#you need to pass n_comps=10 in diffmap() in order to exactly reproduce previous dpt() results. 
sc.tl.diffmap(adata, n_comps=10),
                  #neighbors_key=None, random_state=0, copy=False)

sc.tl.dpt(adata)#, n_dcs=10, n_branchings=0, 
              #min_group_size=0.01, allow_kendall_tau_shift=True, 
              #neighbors_key=None, copy=False)


sc.pl.draw_graph(adata, color=['annot', 'dpt_pseudotime'], legend_loc='on data', legend_fontsize= 'x-small')

## 3D UMAP changing n_neighbor values
based on https://github.com/MNoichl/UMAP-examples-mammoth-/tree/master

In [ ]:
import pandas as pd
import numpy as np
import umap
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
#For Tables:
from IPython.display import display
from IPython.display import Latex
pd.set_option('display.max_columns', 500)
from tqdm import tnrange, tqdm_notebook
%load_ext rpy2.ipython

     


In [ ]:
#3D coordinates
sc.tl.umap(adata_downsampled, n_components=3)
sn_umap = adata_downsampled.obsm['X_umap']
adata.obsm['X_umap']

In [ ]:
#plot the umap
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(90,72))

ax = fig.add_subplot(111, projection='3d')
# ax.set_aspect('equal')

ax.set_axis_off()
ax.scatter(sn_umap[:,0], sn_umap[:,1], sn_umap[:,2], s=20,c='black')
ax.view_init(0, -170)

plt.show()

#works

In [ ]:
reducer = umap.UMAP(random_state=42,
                    init='random',
                    n_components=3,
                    n_neighbors=300,
                    min_dist=0.1,
                    spread=2,
                    metric='euclidean',
                    verbose=True)

reducer = reducer.fit(sn_umap)
#works

In [ ]:
plt.figure(figsize=(40,40),facecolor='w')


#fig, ax = plt.subplots(facecolor='w')
plt.axis('off')
plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5,c='black')

#ax = plt.Axes(fig, [0., 0., 1., 1.])
#ax.set_axis_off()

In [ ]:
fig = plt.figure(figsize=(40,40),facecolor='w')
ax = plt.subplots(facecolor='w')
plt.axis('off')

#2D
#plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5, c='black')

#3D
plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], reducer.embedding_[:, 2], #s=2.5, 
            c='black')


#ax = plt.Axes(fig, [0., 0., 1., 1.])
#ax.set_axis_off()
##works

In [ ]:
# Perform agglomerative clustering on a subset, then scale up with a classifier. Courtesy sb. on StackOverflow. 
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters=15).fit(sn_umap) #change to however many clusters you're working with 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = \
    train_test_split(sn_umap, sn_umap, 
                     test_size=6000, random_state=22) 
AC = AgglomerativeClustering(n_clusters=15, linkage='ward')
AC.fit(X_train)
labels = AC.labels_

KN = KNeighborsClassifier(n_neighbors=10)
KN.fit(X_train,labels)
labels2 = KN.predict(sn_umap)
col_len = len(set(labels2))-1

#works

In [ ]:
%%R -i col_len -o color_scale

mycolors <- c('#1f77b4',
 '#ff7f0e',
 '#279e68',
 '#d62728',
 '#aa40fc',
 '#8c564b',
 '#e377c2',
 '#b5bd61',
 '#17becf',
 '#aec7e8',
 '#ffbb78',
 '#98df8a',
 '#ff9896',
 '#c5b0d5',
 '#c49c94')
pal <- colorRampPalette(sample(mycolors))
color_scale <- sample(pal(col_len))
color_scale <- c(color_scale)

In [ ]:
cmap = mpl.colors.ListedColormap(list(color_scale))
sns.palplot (color_scale)

In [ ]:
cmap = mpl.colors.ListedColormap(adata_downsampled.uns['leiden_1.0_colors'])
cmap
#sns.palplot(color_scale)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(48,35))

ax = fig.add_subplot(111, projection='3d')
# ax.set_aspect('equal')

ax.set_axis_off()
ax.scatter(sn_umap[:,0], sn_umap[:,1], sn_umap[:,2], s=5, c=labels2, cmap=cmap)
ax.view_init(10, -170)

plt.show()

In [ ]:
cmap

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(48,30))

ax = fig.add_subplot(111, projection='3d')
# ax.set_aspect('equal')

ax.set_axis_off()
ax.scatter(sn_umap[:,0], sn_umap[:,1], sn_umap[:,2], s=5, c=labels2, cmap=cmap)
ax.view_init(90, 0)

plt.show()

In [ ]:
plt.figure(figsize=(40,40),facecolor='w')

plt.axis('off')
plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=5,c=labels2,cmap=cmap)

In [ ]:
#Changing n_neighbours
#Loop over several values of n_neighbours and always use the previous embedding as the initialization of the new one instead of a spectral embedding. Slightly reduce the learning rate, so that this works nicely.

neighbours = list(range(5,120,5))
neighbours.reverse()
first = True
coordinates = []
for n in tqdm_notebook(neighbours):
    #min_dist=0.1,

    if first == True:
        first = False
        reducer = umap.UMAP(n_components=2,n_neighbors=n,
                        metric='euclidean')
        reducer.fit(sn_umap)
    else:
        reducer = umap.UMAP(n_components=2,n_neighbors=n,
                        metric='euclidean',learning_rate=0.9,init=embedding)
        reducer.fit(sn_umap)
    embedding = reducer.embedding_

    plt.figure(figsize=(6,6),facecolor='w')
    plt.axis('off')
    plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5,c=labels2,cmap=cmap)#color='black' doesnt need labesl2
    emb = pd.DataFrame(embedding)
    emb.columns = ['x','y']
    emb["n"] = n
    coordinates.append(emb)

In [ ]:
coordinates_full_df = pd.concat(coordinates, ignore_index=True)

In [ ]:
%%R -i coordinates_full_df
#--width 4900 --height 3200 -r 140 --bg #ffffff
library(hrbrthemes)
library(ggplot2)
library(fields)
library(akima)
library(directlabels)
library(zoo)
library(gridExtra)
library(plotly)

library(ggalt)
library(mgcv)
library(scales)
library(gganimate)
library(gifski)
library(png)

timespan <- length(unique(coordinates_full_df$n))

p <- ggplot(data = coordinates_full_df,aes(x=x,y=y))+#,group=id,color=as.factor(journal)))+#as.factor(id)))+#))+
# geom_point(data=subset(coordinates_full_df, cluster == -1), aes(x=x, y=y),pch=16,cex=1.5,alpha=0.2, color=myGray)+                      
# geom_point(data=subset(coordinates_full_df, cluster != -1), aes(x=x, y=y,color=as.factor(cluster)),pch=16,cex=1.5,alpha=0.6)+                      

geom_point(size = 0.9, stroke = 0,alpha=1)+

theme_ipsum_rc()+
theme(legend.position = "none")+
labs(x="", y="",
       title="UMAP of a Woolly Mammoth",
#        subtitle="...based on the code by McInnes, Healy (2018)",
       caption="by Maximilian Noichl, UMAP by McInnes, Healy (2018), Mammooth by Smithsonian 3D")+
theme(panel.grid.major = element_line(colour = "grey", linetype="dotted", size=0.55),panel.grid.minor = element_blank())+
 theme(plot.background = element_rect(fill = "#fbf8f1"))+

  transition_states(n) +
  ease_aes('sine-in-out', interval = 0.01)+#,enter_length = 0.002,exit_length = 0.002)+,nframes = 1000
enter_appear(early = FALSE)+
#  enter_fade()+
#    exit_fade()+
exit_disappear(early = TRUE)+
labs(subtitle = "Embedding for n_neighbours = {closest_state}")

# p <- p + 
#   transition_time(year,transition_length = 4,state_length = 0)+
#   labs(title = "Year: {frame_time}")
p <-animate(p, 
        duration = timespan,#s/100, # = 365 days/yr x 3 years x 0.25 sec/day = 274 seconds
        fps  =  28,width=1000,height=1000)#,nframes = [...or pick it here])



print(p)
anim_save('MWB_anim_nearest_neighbours.gif', animation = p)

In [ ]:
#Do the same for min distances

min_dists = [0.001,0.005,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
min_dists.reverse()
min_dists
first = True
coordinates = []
for d in tqdm_notebook(min_dists):
    #min_dist=0.1,

    if first == True:
        first = False
        reducer = umap.UMAP(n_components=2,n_neighbors=100,min_dist=d,
                        metric='euclidean')
        reducer.fit(sn_umap)
    else:
        reducer = umap.UMAP(n_components=2,n_neighbors=100,min_dist=d,
                        metric='euclidean',learning_rate=0.9,init=embedding)#,learning_rate=0.9
        reducer.fit(sn_umap)
    embedding = reducer.embedding_

    plt.figure(figsize=(6,6),facecolor='w')
    plt.axis('off')
    #plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5,color='black')
    plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5,c=labels2,cmap=cmap)

    emb = pd.DataFrame(embedding)
    emb.columns = ['x','y']
    emb["d"] = d
    coordinates.append(emb)

In [ ]:
coordinates_full_df = pd.concat(coordinates, ignore_index=True)

In [ ]:
%%R -i coordinates_full_df
#--width 4900 --height 3200 -r 140 --bg #ffffff
library(hrbrthemes)
library(ggplot2)
library(fields)
library(akima)
library(directlabels)
library(zoo)
library(gridExtra)
library(plotly)

library(ggalt)
library(mgcv)
library(scales)
library(gganimate)
library(gifski)
library(png)

timespan <- length(unique(coordinates_full_df$d))

p <- ggplot(data = coordinates_full_df,aes(x=x,y=y))+#,group=id,color=as.factor(journal)))+#as.factor(id)))+#))+
# geom_point(data=subset(coordinates_full_df, cluster == -1), aes(x=x, y=y),pch=16,cex=1.5,alpha=0.2, color=myGray)+                      
# geom_point(data=subset(coordinates_full_df, cluster != -1), aes(x=x, y=y,color=as.factor(cluster)),pch=16,cex=1.5,alpha=0.6)+                      

geom_point(size = 0.9, stroke = 0,alpha=1)+

theme_ipsum_rc()+
theme(legend.position = "none")+
labs(x="", y="",
       title="UMAP of a Woolly Mammoth",
        subtitle="...based on the code by McInnes, Healy (2018)",
       caption="by Maximilian Noichl, UMAP by McInnes, Healy (2018), Mammooth by Smithsonian 3D")+
#theme(panel.grid.major = element_line(colour = "grey", linetype="dotted", size=0.55),panel.grid.minor = element_blank())+
 theme(plot.background = element_rect(fill = "#fbf8f1"))+

  transition_states(d) +
  ease_aes('sine-in-out', interval = 0.01)+#,enter_length = 0.002,exit_length = 0.002)+,nframes = 1000
enter_appear(early = FALSE)+
  enter_fade()+
    exit_fade()+
exit_disappear(early = TRUE)+
labs(subtitle = "Embedding for min_dist = {closest_state}")

# p <- p + 
#   transition_time(year,transition_length = 4,state_length = 0)+
#   labs(title = "Year: {frame_time}")
p <- animate(p, 
        duration = timespan,#s/100, # = 365 days/yr x 3 years x 0.25 sec/day = 274 seconds
        fps  =  28,width=1000,height=1000)#,nframes = [...or pick it here])
print(p)
anim_save('MWBs_dist_param.gif', animation = p)

In [ ]:
#Do the same for random states

reducer = umap.UMAP(#random_state=42,
                     init='random',
                    n_components=3,
                    n_neighbors=300,
                    min_dist=0.1,
#                     spread=2,
                    metric='euclidean',
                    verbose=True)

reducer = reducer.fit(sn_umap)
#works

first = True
coordinates = []
for d in tqdm_notebook(min_dists):
    #min_dist=0.1,

    if first == True:
        first = False
        reducer = umap.UMAP(n_components=2,n_neighbors=100,min_dist=.1, #keep umap constant
                        metric='euclidean')
        reducer.fit(sn_umap)
    else:
        reducer = umap.UMAP(n_components=2,n_neighbors=100,min_dist=.1,
                        metric='euclidean',learning_rate=0.9,init=embedding)#,learning_rate=0.9
        reducer.fit(sn_umap)
    embedding = reducer.embedding_

    plt.figure(figsize=(6,6),facecolor='w')
    plt.axis('off')
    #plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5,color='black')
    plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5,c=labels2,cmap=cmap)

    emb = pd.DataFrame(embedding)
    emb.columns = ['x','y']
    emb["d"] = d
    coordinates.append(emb)

In [ ]:
#Another 3D UMAP tutorial from SamBiomics on utube 
#https://www.youtube.com/watch?v=XzAryDJTi1M

#For 3D a
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#For more than 20 colors 
#!pip install distinctipy 
#https://distinctipy.readthedocs.io/en/latest/usage.html#Import-distinctipy
import distinctipy
from distinctipy import colorblind
from distinctipy import examples

import matplotlib.pyplot as plt

In [ ]:
type(adata_downsampled.uns['leiden_1.0_colors'])

In [ ]:
type(plt.cm.tab20(range(0,15)))

In [ ]:
color = dict(zip(range(0,10), adata_downsampled.uns['leiden_1.0_colors']))

In [ ]:
plt.cm.tab20(range(0,10))

In [ ]:
sc.tl.umap(adata_downsampled, n_components=3)
umap = adata_downsampled.obsm['X_umap']
umap

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(projection = '3d')

ax.scatter(umap[:,0], umap[:,1], umap[:,2])

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(projection = '3d')

ax.scatter(umap[:,0],umap[:,1],umap[:,2], 
           c = adata_downsampled.obs['leiden_1.0'].astype('int').map(color))

plt.show()

In [ ]:
!mkdir figs
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(projection = '3d')

ax.scatter(umap[:,0], umap[:,1], umap[:,2], 
           c = adata_downsampled.obs['leiden_1.0'].astype('int').map(color))

x_center = (umap[:,0].max() + umap[:,0].min())/2
y_center = (umap[:,1].max() + umap[:,1].min())/2
z_center = (umap[:,2].max() + umap[:,2].min())/2

ax.plot([x_center,x_center], [y_center, y_center], [umap[:,2].min() - 2, umap[:,2].max() + 2], c = 'k', lw = 5)
ax.plot([x_center,x_center], [umap[:,1].min() - 2, umap[:,1].max() + 2], [z_center, z_center], c = 'k', lw = 5)
ax.plot([umap[:,0].min() - 2, umap[:,0].max() + 2], [y_center, y_center], [z_center, z_center], c = 'k', lw = 5)
    
ax.axis('off')

plt.savefig(f'figs/{i:003}', dpi = 100, facecolor = '#fbf8f1')

plt.show()

In [ ]:
for i in range(0, 360, 2):
    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(projection = '3d')
    
    ax.scatter(umap[:,0],umap[:,1],umap[:,2], 
               c = adata_downsampled.obs['leiden_1.0'].astype('int').map(color))
    #position
    x_center = (umap[:,0].max() + umap[:,0].min())/2
    y_center = (umap[:,1].max() + umap[:,1].min())/2
    z_center = (umap[:,2].max() + umap[:,2].min())/2

    #lines
    ax.plot([x_center,x_center], [y_center, y_center], [umap[:,2].min() - 2, umap[:,2].max() + 2], c = 'k', lw = 5)
    ax.plot([x_center,x_center], [umap[:,1].min() - 2, umap[:,1].max() + 2], [z_center, z_center], c = 'k', lw = 5)
    ax.plot([umap[:,0].min() - 2, umap[:,0].max() + 2], [y_center, y_center], [z_center, z_center], c = 'k', lw = 5)
    
    ax.view_init(20, i)
    
    ax.axis('off')

    plt.savefig(f'figs/{i:003}', dpi = 100, facecolor = '#fbf8f1')

    #plt.legend()
    plt.show()

In [ ]:
!magick -delay 5 figs/*.png MWBs_leiden_1.0.gif